In [8]:
import time
import tracemalloc
import functools
import threading
import psutil
import os
import random

In [10]:
def timeit(func):
    """
    Декоратор для измерения времени работы функции.
    Суммирует время всех рекурсивных вызовов (если функция сама себя вызывает).
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        if not hasattr(wrapper, "_time_acc"):
            wrapper._time_acc = 0.0
        start = time.perf_counter()
        try:
            return func(*args, **kwargs)
        finally:
            end = time.perf_counter()
            wrapper._time_acc += (end - start)
            # если это внешний вызов (не рекурсивный), то печатаем и сбрасываем
            # мы определим «внешний вызов» как момент, когда стек глубже, чем вызов функции
            wrapper._recursion_depth -= 1
            if wrapper._recursion_depth == 0:
                total = wrapper._time_acc
                # сброс для следующего полного запуска
                wrapper._time_acc = 0.0
                print(f"[timeit] Function {func.__name__} took total {total:.6f} seconds")
        # Для корректного определения рекурсии:

    def new_wrapper(*args, **kwargs):
        if not hasattr(wrapper, "_recursion_depth"):
            wrapper._recursion_depth = 0
        wrapper._recursion_depth += 1
        return wrapper(*args, **kwargs)

    return new_wrapper


def memoryit(func):
    """
    Декоратор для измерения пикового использования памяти Python во время работы функции.
    Использует tracemalloc для Python-памяти + psutil для общего RSS процесса.
    Аккумулирует пиковые значения в рекурсии.
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # инициализация
        if not hasattr(wrapper, "_mem_peak_python"):
            wrapper._mem_peak_python = 0
            wrapper._mem_peak_rss = 0
        if not hasattr(wrapper, "_call_depth"):
            wrapper._call_depth = 0

        if wrapper._call_depth == 0:
            # первый (внешний) вызов — запускаем трассировку
            tracemalloc.start()
        wrapper._call_depth += 1

        rss_before = psutil.Process(os.getpid()).memory_info().rss

        try:
            return func(*args, **kwargs)
        finally:
            current, peak = tracemalloc.get_traced_memory()
            rss_after = psutil.Process(os.getpid()).memory_info().rss

            # обновляем пик Python-аллокатора
            if peak > wrapper._mem_peak_python:
                wrapper._mem_peak_python = peak
            # обновляем пик RSS
            rss_delta = rss_after - rss_before
            if rss_delta > wrapper._mem_peak_rss:
                wrapper._mem_peak_rss = rss_delta

            wrapper._call_depth -= 1
            if wrapper._call_depth == 0:
                # завершение внешнего вызова
                tracemalloc.stop()
                # вывод результатов
                print(
                    f"[memoryit] Function {func.__name__} peak python allocation: {wrapper._mem_peak_python / 1024:.2f} KiB")
                print(f"[memoryit] Function {func.__name__} peak RSS delta: {wrapper._mem_peak_rss / 1024:.2f} KiB")
                # сброс для следующего вызова
                wrapper._mem_peak_python = 0
                wrapper._mem_peak_rss = 0

    return wrapper

In [12]:
Задача 3. Симметрическая разность.
Ограничение по времени: 2 с. Ограничение по памяти: 64 Mb.
На вход подается множество чисел в диапазоне от 1 до 20000, разделенных
пробелом. Они образуют множество А. Затем идет разделитель – число 0 и на вход подается
множество чисел В, разделенных пробелом, 0 – признак конца описания множества (во
множество не входит). Необходимо вывести множество АΔВ – симметрическую разность
множеств А и В в порядке возрастания элементов. В качестве разделителя используйте
пробел. В случае, если множество пусто, вывести 0.
Формат входных данных:
1 2 3 4 5 0 1 7 5 8 0
Формат выходных данных:
2 3 4 7 8

SyntaxError: invalid character '–' (U+2013) (2295159655.py, line 4)

In [14]:

def _selection_sort(items: list[int]) -> None:
    n = len(items)
    for i in range(n):
        min_idx = i
        for j in range(i + 1, n):
            if items[j] < items[min_idx]:  # ищем самый маленький
                min_idx = j
        items[i], items[min_idx] = items[min_idx], items[i]


def _join_numbers(items: list[int]) -> str:
    if not items:
        return ""
    result = str(items[0])
    for value in items[1:]:
        result += " " + str(value)
    return result


@timeit
@memoryit
def symmetric_difference(input_str: str) -> str:
    set_a: list[int] = []
    set_b: list[int] = []
    result: list[int] = []
    input_list = input_str.split()

    is_b = False

    for symbol in input_list:
        number = int(symbol)
        if number == 0:
            if is_b:
                break
            is_b = True
            continue
        if not is_b:
            set_a.append(number)
        else:
            set_b.append(number)

    for value in set_a:
        if value not in set_b:
            result.append(value)

    for value in set_b:
        if value not in set_a:
            result.append(value)

    _selection_sort(result)

    if not result:
        return "0"
    return _join_numbers(result)


print(symmetric_difference('1500 7800 12345 567 8900 4321 9999 150 6789 3210 2555 7777 11111 5432 8888 6666 4444 2222 3333 5555 0 7800 567 8900 20000 150 123 4567 9999 4321 6789 3210 11111 8888 6666 4444 0'))
print(symmetric_difference('100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 0 50 150 250 350 450 550 650 750 850 950 1050 1150 1250 1350 1450 1550 1650 1750 1850 1950 2050 2150 2250 2350 2450 0'))
print(symmetric_difference('42 1984 2023 777 1337 69 420 1234 5678 9999 11111 15555 17777 19999 2468 13579 8642 7531 2222 3333 4444 5555 6666 7777 8888 0 42 777 1337 10000 15000 20000 2468 8642 2222 4444 6666 8888 11111 15555 17777 19999 0'))
print(symmetric_difference('1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 0 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 0'))
print(symmetric_difference('15000 2500 7500 12500 500 17500 1000 20000 3000 4000 6000 7000 8000 9000 11000 12000 13000 14000 16000 17000 18000 19000 0 2500 7500 12500 17500 333 666 999 1333 1666 1999 2333 2666 2999 3333 3666 3999 4333 4666 4999 0'))
print(symmetric_difference('111 222 333 444 555 666 777 888 999 1111 2222 3333 4444 5555 6666 7777 8888 9999 11111 12222 13333 14444 15555 16666 17777 18888 19999 0 222 444 666 888 1111 3333 5555 7777 9999 12222 14444 16666 18888 20000 12345 13579 14789 15987 0'))
print(symmetric_difference('123 456 789 1011 1213 1415 1617 1819 2021 2223 2425 2627 2829 3031 3233 3435 3637 3839 4041 4243 4445 4647 4849 5051 5253 5455 5657 5859 6061 6263 0 456 789 1011 1415 1819 2223 2627 3031 3435 3839 4243 4647 5051 5455 5859 6263 6666 7777 8888 9999 11111 0'))
print(symmetric_difference('5000 10000 15000 2000 4000 6000 8000 12000 14000 16000 18000 2500 3500 4500 5500 6500 7500 8500 9500 10500 11500 12500 13500 14500 15500 16500 17500 18500 19500 0 2000 4000 6000 8000 10000 12000 14000 16000 18000 2500 4500 6500 8500 10500 12500 14500 16500 18500 19500 333 666 999 0'))
print(symmetric_difference('11111 12222 13333 14444 15555 16666 17777 18888 19999 20000 1111 2222 3333 4444 5555 6666 7777 8888 9999 1234 2345 3456 4567 5678 6789 7890 8901 9012 1357 2468 3579 4680 5791 6802 7913 8024 9135 0 12222 14444 16666 18888 20000 2222 4444 6666 8888 2345 4567 6789 8901 2468 4680 6802 8024 10000 15000 0'))
print(symmetric_difference('19999 20000 1 2 3 4 5 6 7 8 9 10 100 200 300 400 500 600 700 800 900 1000 1111 2222 3333 4444 5555 6666 7777 8888 9999 12345 13579 14999 15999 16999 17999 18999 0 1 3 5 7 9 100 300 500 700 900 1111 3333 5555 7777 9999 12345 14999 15999 17999 18999 20000 42 142 242 342 442 0'))

[memoryit] Function symmetric_difference peak python allocation: 4.20 KiB
[memoryit] Function symmetric_difference peak RSS delta: 0.00 KiB
[timeit] Function symmetric_difference took total 0.001754 seconds
123 1500 2222 2555 3333 4567 5432 5555 7777 12345 20000
[memoryit] Function symmetric_difference peak python allocation: 5.56 KiB
[memoryit] Function symmetric_difference peak RSS delta: 4.00 KiB
[timeit] Function symmetric_difference took total 0.000719 seconds
50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1100 1150 1200 1250 1300 1350 1400 1450 1500 1550 1600 1650 1700 1750 1800 1850 1900 1950 2000 2050 2100 2150 2200 2250 2300 2350 2400 2450 2500
[memoryit] Function symmetric_difference peak python allocation: 4.07 KiB
[memoryit] Function symmetric_difference peak RSS delta: 0.00 KiB
[timeit] Function symmetric_difference took total 0.000341 seconds
69 420 1234 1984 2023 3333 5555 5678 7531 7777 9999 10000 13579 15000 20000
[memoryit] Functi

In [16]:
Задача 5. Длинное сложение и вычитание
Ограничение по времени: 2 с. Ограничение по памяти: 64 Mb.
На вход подается три строки. Первая содержит представление длинного десятичного
числа (первый операнд), вторая – представление операции, строки + и -, третья –
представление второго операнда.
Длина первой и третьей строки ограничены 1000 символами. Вторая строка
содержит ровно один символ.
Требуется исполнить операцию и вывести результат в десятичном представлении.
Формат входных данных:
123
+
999
Формат выходных данных:
1122

SyntaxError: invalid character '–' (U+2013) (1110372328.py, line 4)

In [18]:
@timeit
@memoryit
def long_a_s(first_operand: str, operator: str, second_operand: str) -> str:
    left = int(first_operand)
    right = int(second_operand)
    if operator == "+":
        return str(left + right)
    if operator == "-":
        return str(left - right)


print(long_a_s("123", "+", "999"))
print(long_a_s("1000", "-", "1"))
print(long_a_s("-50", "+", "25"))
print(long_a_s("500", "-", "1234"))


[memoryit] Function long_a_s peak python allocation: 1.35 KiB
[memoryit] Function long_a_s peak RSS delta: 0.00 KiB
[timeit] Function long_a_s took total 0.000296 seconds
1122
[memoryit] Function long_a_s peak python allocation: 1.35 KiB
[memoryit] Function long_a_s peak RSS delta: 0.00 KiB
[timeit] Function long_a_s took total 0.000189 seconds
999
[memoryit] Function long_a_s peak python allocation: 1.35 KiB
[memoryit] Function long_a_s peak RSS delta: 0.00 KiB
[timeit] Function long_a_s took total 0.000130 seconds
-25
[memoryit] Function long_a_s peak python allocation: 1.35 KiB
[memoryit] Function long_a_s peak RSS delta: 0.00 KiB
[timeit] Function long_a_s took total 0.000128 seconds
-734


In [20]:
Задача 8. Магараджа.
Ограничение по времени: 1 с. Ограничение по памяти: 16 Mb.
Магараджа — это шахматная фигура, сочетающая возможности ферзя и коня. Таким
образом, магараджа может ходить и бить на любое количество клеток по диагонали,
горизонтали и вертикали (т.е. как ферзь), а также либо на две клетки по горизонтали и на
одну по вертикали, либо на одну по горизонтали и на две по вертикали (как конь).
Ваша задача — найти число способов расставить на доске N на N ровно K
магараджей так, чтобы они не били друг друга.

SyntaxError: invalid character '—' (U+2014) (752038603.py, line 3)

In [40]:
def maharajah_attacks(coord1, coord2):
    r1, c1 = coord1
    r2, c2 = coord2
    
    # Атака Ферзя
    if r1 == r2 or c1 == c2 or abs(r1 - r2) == abs(c1 - c2):
        return True
    
    # Атака Коня
    dr = abs(r1 - r2)
    dc = abs(c1 - c2)
    if (dc == 2 and dr == 1) or (dc == 1 and dr == 2):
        return True
        
    return False

def solve_maharajah(n, k):
    if n == 0:
        return 1 if k == 0 else 0
    if k == 0:
        return 1
    if k == 1:
        return n * n

    cells = [(r, c) for r in range(n) for c in range(n)]
    total_cells = len(cells)
    
    if k > total_cells:
        return 0
        
    count = 0

    def back_track(idx, placed_pieces):
        nonlocal count
        
        if len(placed_pieces) == k:
            count += 1
            return
            
        if idx >= total_cells or len(placed_pieces) + (total_cells - idx) < k:
            return
            
        # 1. Пропустить cells[idx]
        back_track(idx + 1, placed_pieces)
        
        # 2. Попробовать разместить cells[idx]
        candidate = cells[idx]
        is_safe = True
        
        for placed_piece in placed_pieces:
            if maharajah_attacks(candidate, placed_piece):
                is_safe = False
                break
                
        if is_safe:
            placed_pieces.append(candidate)
            back_track(idx + 1, placed_pieces)
            placed_pieces.pop()

    back_track(0, [])
    return count

def run_test(name, n, k, expected):
    result = solve_maharajah(n, k)
    status = "✓ ПРОЙДЕН" if result == expected else f"✗ ПРОВАЛЕН (Ожидали: {expected}, Получили: {result})"
    print(f"\n{name} (N={n}, K={k})")
    print(f"Статус: {status}")


print("=== НАЧАЛО ТЕСТИРОВАНИЯ МАГАРАДЖИ ===")

# Корректные тесты для Магараджи
run_test("Тест 1: N=1, K=1", 1, 1, 1)
run_test("Тест 2: N=2, K=1", 2, 1, 4)
run_test("Тест 3: N=3, K=1", 3, 1, 9)
run_test("Тест 4: N=2, K=2", 2, 2, 0) # Невозможно разместить
run_test("Тест 5: N=3, K=2", 3, 2, 0) # Невозможно разместить

# ИСПРАВЛЕННЫЕ тесты для N=4
run_test("Тест 6: N=4, K=2", 4, 2, 20) 
run_test("Тест 7: N=4, K=3", 4, 3, 0) # Невозможно разместить

print("\n=== ТЕСТИРОВАНИЕ ЗАВЕРШЕНО ===")

=== НАЧАЛО ТЕСТИРОВАНИЯ МАГАРАДЖИ ===

Тест 1: N=1, K=1 (N=1, K=1)
Статус: ✓ ПРОЙДЕН

Тест 2: N=2, K=1 (N=2, K=1)
Статус: ✓ ПРОЙДЕН

Тест 3: N=3, K=1 (N=3, K=1)
Статус: ✓ ПРОЙДЕН

Тест 4: N=2, K=2 (N=2, K=2)
Статус: ✓ ПРОЙДЕН

Тест 5: N=3, K=2 (N=3, K=2)
Статус: ✓ ПРОЙДЕН

Тест 6: N=4, K=2 (N=4, K=2)
Статус: ✓ ПРОЙДЕН

Тест 7: N=4, K=3 (N=4, K=3)
Статус: ✓ ПРОЙДЕН

=== ТЕСТИРОВАНИЕ ЗАВЕРШЕНО ===


In [38]:
Задача 11. Поиск в сломанном массиве
Ограничение по времени: 0.001 с. Ограничение по памяти: 64 Mb.
Алла ошиблась при копировании из одной структуры данных в другую. Она хранила
массив чисел в кольцевом буфере. Массив был отсортирован по возрастанию, и в нем
можно было найти элемент за логарифмическое время. Алла скопировала данные из
кольцевого буфера в обычный массив, но сдвинула данные исходной отсортированной
последовательности. Теперь массив не является отсортированным Тем не менее, нужно
обеспечить возможность находить внем элемент за О(log n).
Можно предполагать, что в массиве только уникальные элементы.
Задачу необходимо сдавать, выбрав компилятор Make! Решение отправляется
файлом. Требуемые сигнатуры функций лежат в заготовках кода на диске.
От вас требуется реализовать функцию, осуществляющую поиск в сломанном
массиве. Файлы с заготовками кода, содержащими сигнатуры Функций и базовый тест для
поддерживаемых языков, находятся на Яндекс Диске по ссылке. Обратите внимание, что
считывать данные и выводить ответ не требуется
Разрешение файла должно соответствовать языку на котором вы пишете (.cpp, .java,
.go, .js, .py). Если вы пишете на Java, назовите файл с решением Solution.java, для C# —
Solution.cs. Для остальных языков название может быть любым, кроме solution.ext где ext
— разрешение для вашего языка.
Формат входных данных:
Функция принимает массив натуральных чисел и искомое число k. Длина массива
не превосходит 10000. Элементы массива и число k не превосходят по значению 10000.
В примерах:
В первой строке записано число n — длина массива.
Во второй строке записано положительное число k — искомый элемент
Далее в строку через пробел записано n натуральных чисел. каждое из которых не
превосходит 200000
Формат выходных данных:
Функция должна вернуть индекс элемента, равного k, если такой есть в массиве
(нумерация с нуля). Если элемент не найден, функция должна вернуть — 1.
Изменять массив нельзя.
Для отсечения неэффективных решений ваша функция будет запускаться от 100000
до 1000000 раз.

SyntaxError: invalid character '—' (U+2014) (2704637133.py, line 19)

In [42]:
@timeit
@memoryit
def broken_search(target: int, nums: str) -> int:
    target = str(target)

    try:
        return nums.index(target)
    except ValueError:
        return -1


numbers = [str(random.randint(0, 200000)) for _ in range(10000)]
nums_str = " ".join(numbers)
print(broken_search(random.randint, nums_str))

[memoryit] Function broken_search peak python allocation: 1.35 KiB
[memoryit] Function broken_search peak RSS delta: 0.00 KiB
[timeit] Function broken_search took total 0.000330 seconds
-1


In [ ]:
Задача 14. Сортировка по многим полям
Ограничение по времени: 2 с. Ограничение по памяти: 64 Mb.
В базе данных хранится N записей, вида (Name, a1, a2, ..., ak) – во всех записях
одинаковое число параметров. На вход задачи подается приоритет полей – перестановка на
числах 1, ..., k – записи нужно вывести по невозрастанию в соответствии с этим
приоритетом. В случае, если приоритет полей таков: 3 4 2 1, то это следует воспринимать
так: приоритет значений из 3 колонки самый высокий, приоритет значений из колонки 4
ниже, приоритет значений из колонки 2 еще ниже, а приоритет значений из колонки 1
самый низкий.
Формат входных данных:
N ≤ 1000
k: 1 ≤ k ≤ 10
p1 p2 ... pk – перестановка на k числах, разделитель – пробел
N строк вида
Name a1 a2 ... ak
Формат выходных данных:
N строк с именами в порядке, согласно приоритету

In [44]:
@timeit
@memoryit
def _selection_sort(items: list, priorities: list) -> None:
    n = len(items)
    for i in range(n):
        min_idx = i
        for j in range(i + 1, n):
            need_swap = False
            for p in priorities:
                idx = int(p) - 1
                if items[j][1][idx] > items[min_idx][1][idx]:
                    need_swap = True
                    break
                elif items[j][1][idx] < items[min_idx][1][idx]:
                    break
            if need_swap:
                min_idx = j
        items[i], items[min_idx] = items[min_idx], items[i]

test_data = [
    (["3", "3", "2 1 3", "A 1 2 3", "B 3 2 1", "C 3 1 2"], ["B", "A", "C"]),
    (["3", "3", "1 2 3", "X 5 1 1", "Y 3 9 9", "Z 4 0 0"], ["X", "Z", "Y"]),
    (["3", "1", "1", "One 10", "Two 30", "Three 20"], ["Two", "Three", "One"])
]

print("=== ТЕСТЫ ===")
for i, (inputs, expected) in enumerate(test_data, 1):
    n = int(inputs[0])
    k = int(inputs[1])
    priorities = inputs[2].split()

    records = []
    for j in range(3, 3 + n):
        data = inputs[j].split()
        name = data[0]
        values = [int(x) for x in data[1:]]
        records.append((name, values))

    _selection_sort(records, priorities)
    result = [name for name, _ in records]

    status = "✅ ПРОЙДЕН" if result == expected else "❌ НЕ ПРОЙДЕН"
    print(f"Тест {i}: {status}")
    print(f"Ожидалось: {expected}")
    print(f"Получилось: {result}\n")

=== ТЕСТЫ ===
[memoryit] Function _selection_sort peak python allocation: 1.35 KiB
[memoryit] Function _selection_sort peak RSS delta: 0.00 KiB
[timeit] Function _selection_sort took total 0.000275 seconds
Тест 1: ✅ ПРОЙДЕН
Ожидалось: ['B', 'A', 'C']
Получилось: ['B', 'A', 'C']

[memoryit] Function _selection_sort peak python allocation: 1.35 KiB
[memoryit] Function _selection_sort peak RSS delta: 0.00 KiB
[timeit] Function _selection_sort took total 0.000146 seconds
Тест 2: ✅ ПРОЙДЕН
Ожидалось: ['X', 'Z', 'Y']
Получилось: ['X', 'Z', 'Y']

[memoryit] Function _selection_sort peak python allocation: 1.35 KiB
[memoryit] Function _selection_sort peak RSS delta: 0.00 KiB
[timeit] Function _selection_sort took total 0.000137 seconds
Тест 3: ✅ ПРОЙДЕН
Ожидалось: ['Two', 'Three', 'One']
Получилось: ['Two', 'Three', 'One']



In [ ]:
Задача 16. Очень быстрая сортировка.
Ограничение по времени: 1.5 с. Ограничение по памяти: 512 Mb.
Имеется рекуррентная последовательность A1, A2, ..., AN, строящаяся по
следующему правилу:
A1 = K
Ai+1 = (Ai × M) % (232
– 1) % L

Требуется найти сумму всех нечетных по порядку элементов в отсортированной по
неубыванию последовательности по модулю L.
Для входных данных
5 7 13 100
последовательность будет такой:
{7; 7 × 13%100 = 91; 91 × 13%100 = 83; 83 × 13%100 = 79; 79 × 13%100 = 27}, то есть
{10; 91; 83; 79; 27}.
Отсортированная последовательность {7; 27; 79; 83; 91}.
Сумма элементов на нечетных местах = (7 + 79 + 91)%100 = 77.
Формат входных данных:
N K M L
5000000 ≤ N ≤ 60000000, 0 ≤ K, L, M ≤ 232
– 1

Формат выходных данных:
RESULT

In [46]:
@timeit
@memoryit
def very_fast_sort(N, K, M, L):

    if L == 0:  # если L = 0, все элементы будут 0
        print(0)
        return

    count = [0] * L  # массив для подсчета количества каждого элемента

    current = K  # генерируем последовательность и считаем элементы
    count[current % L] += 1  # генерируем последовательность и считаем элементы

    for i in range(1, N):
        current = (current * M) % (4294967295) % L
        count[current] += 1

    # собираем отсортированную последовательность и считаем сумму нечетных позиций
    result = 0
    pos = 1  # текущая позиция в отсортированном массиве

    for num in range(L):
        for j in range(count[num]):
            if pos % 2 == 1:  # нечетная позиция
                result = (result + num) % L
            pos += 1

    print(result)


print("Тест 1:", very_fast_sort(5, 7, 13, 100), "(ожидается 77)")
print("Тест 2:", very_fast_sort(3, 5, 1, 10), "(ожидается 0)")
print("Тест 3:", very_fast_sort(1, 42, 13, 100), "(ожидается 42)")
print("Тест 4:", very_fast_sort(3, 1, 2, 10), "(ожидается 5)")
print("Тест 5:", very_fast_sort(4, 0, 5, 10), "(ожидается 0)")

77
[memoryit] Function very_fast_sort peak python allocation: 1.35 KiB
[memoryit] Function very_fast_sort peak RSS delta: 0.00 KiB
[timeit] Function very_fast_sort took total 0.000541 seconds
Тест 1: None (ожидается 77)
0
[memoryit] Function very_fast_sort peak python allocation: 1.35 KiB
[memoryit] Function very_fast_sort peak RSS delta: 0.00 KiB
[timeit] Function very_fast_sort took total 0.000194 seconds
Тест 2: None (ожидается 0)
42
[memoryit] Function very_fast_sort peak python allocation: 1.35 KiB
[memoryit] Function very_fast_sort peak RSS delta: 0.00 KiB
[timeit] Function very_fast_sort took total 0.000422 seconds
Тест 3: None (ожидается 42)
5
[memoryit] Function very_fast_sort peak python allocation: 1.35 KiB
[memoryit] Function very_fast_sort peak RSS delta: 0.00 KiB
[timeit] Function very_fast_sort took total 0.000186 seconds
Тест 4: None (ожидается 5)
0
[memoryit] Function very_fast_sort peak python allocation: 1.35 KiB
[memoryit] Function very_fast_sort peak RSS delta: 0.0

In [ ]:
Задача 20. Аттракцион.
Ограничение по времени: 1 с. Ограничение по памяти: 16 Mb.
На протяжении многих лет в Байтландии существует парк развлечений “Funny byte”,
в котором представлено много различных аттракционов: колесо вычислений, веселые горки
с трассами в форме интегралов, равнобедренная ромашка и многие другие.
В последние годы популярность “Funny byte” стала неуклонно падать. В целях
привлечения посетителей руководство парка решило открыть новый аттракцион, который
представляет собой сложный механизм.
Кресла аттракциона расположены в N рядов по M кресел в каждом. То есть каждое
кресло характеризуется номером ряда и номером колонки, в котором оно находится. Ряды
нумеруются последовательно сверху вниз начиная с единицы, колонки нумеруются слева
направо начиная с единицы. Каждое кресло имеет свой уникальный номер. Кресло,
находящееся в i-м ряду и в j-ой колонке, имеет номер (i-1)*M + j.
После посадки отдыхающих, кресла поднимают вверх над землей. И начинается
веселье! Механизм аттракциона случайным образом производит некоторое количество
операций. Под одной операцией понимается взаимная перестановка двух рядов либо двух
колонок. При взаимной перестановке двух рядов или колонок каждое кресло в ряду или
колонке заменятся на соответствующее ему кресло в другом ряду или колонке.
И вот аттракцион завершил свою работу. Но есть одна трудность! Кресла надо
вернуть в начальное положение при помощи таких же операций. Как количество, так и сами

операции не обязательно должны быть идентичны тем, которые производил аттракцион во
время сеанса. Руководство парка решило, что вернуть кресла в начальное положение
необходимо не более чем за 1000 операций.
Такая задача оказалась не по силам разработчикам механизма. Помогите им! От вас
требуется разработать программу, позволяющую вернуть кресла в начальное положение.
Гарантируется, что решение существует.
Формат входных данных:
В первой строке входного файла INPUT.TXT заданы два натуральных числа N и M
(1 ≤ N, M ≤ 250). В последующих N строках задано по M натуральных чисел, где j-е число
в i+1-й строке соответствует номеру кресла после окончания сеанса аттракциона. Числа в
строках разделяются одиночными пробелами.
Формат выходных данных:
В первой строке выходного файла OUTPUT.TXT должно быть выведено количество
операций перестановки K, которое не должно превышать 1000. Каждая из следующих K
строк описывает одну операцию. Каждая операция описывается строкой вида Q X Y, где Q
– символ 'R'(ASCII 82) либо символ 'C'(ASCII 67). Если Q равно 'R', то данная операция
является перестановкой рядов, если Q равно 'C', то операция является перестановкой
колонок. X и Y – два натуральных числа, соответствующие номерам рядов (колонок),
которые будут переставлены в результате данной операции. Операции должны быть
выведены в порядке осуществления, то есть последовательное применение которых
позволит вернуть кресла в начальное положение.

In [48]:
@timeit
@memoryit
def run_solution(N, M, matrix):
    a = [row[:] for row in matrix]  # копируем матрицу
    ops = []

    # восстановление строк
    for i in range(N):
        need_min = i * M + 1
        found_pos = 0

        for r in range(i, N):
            current_min = a[r][0]
            for val in a[r]:
                if val < current_min:
                    current_min = val
            if current_min == need_min:
                found_pos = r
                break

        if found_pos != i and found_pos != -1:
            ops.append(f"R {i + 1} {found_pos + 1}")
            temp = a[i]
            a[i] = a[found_pos]
            a[found_pos] = temp

    # восстановление столбцов
    for j in range(M):
        need_val = j + 1
        found_pos = 0

        for c in range(j, M):
            if a[0][c] == need_val:
                found_pos = c
                break

        if found_pos != j and found_pos != -1:
            ops.append(f"C {j + 1} {found_pos + 1}")
            for r in range(N):
                temp = a[r][j]
                a[r][j] = a[r][found_pos]
                a[r][found_pos] = temp

    return ops

@timeit
@memoryit
def apply_operations(N, M, matrix, ops):
    # gрименяет операции к матрице и возвращает результат
    a = [row[:] for row in matrix]

    for op in ops:
        parts = op.split()
        type_op = parts[0]
        x = int(parts[1]) - 1
        y = int(parts[2]) - 1

        if type_op == 'R':
            # Меняем строки
            temp = a[x]
            a[x] = a[y]
            a[y] = temp
        else:  # 'C'
            # Меняем столбцы
            for r in range(N):
                temp = a[r][x]
                a[r][x] = a[r][y]
                a[r][y] = temp

    return a

@timeit
@memoryit
def is_correct_final_matrix(N, M, matrix):
    # является ли матрица исходной
    for i in range(N):
        for j in range(M):
            if matrix[i][j] != i * M + j + 1:
                return False
    return True


def print_matrix(matrix):
    for row in matrix:
        print(' '.join(map(str, row)))


def run_test(test_name, N, M, input_matrix):
    print(f"\n=== {test_name} ===")
    print("Входная матрица:")
    print_matrix(input_matrix)

    ops = run_solution(N, M, input_matrix)

    print(f"\nОперации ({len(ops)}):")
    for op in ops:
        print(op)

    # Проверяем корректность
    result_matrix = apply_operations(N, M, input_matrix, ops)
    print("\nРезультирующая матрица:")
    print_matrix(result_matrix)

    is_correct = is_correct_final_matrix(N, M, result_matrix)
    print(f"\nКорректность: {is_correct}")

    return is_correct, ops


# Тест 1: Пример из условия
def test1():
    N, M = 2, 2
    matrix = [
        [4, 3],
        [2, 1]
    ]
    return run_test("Тест 1 (пример из условия)", N, M, matrix)


# Тест 2: Уже правильная матрица
def test2():
    N, M = 3, 5
    matrix = [
        [1, 2, 3, 4, 5],
        [6, 7, 8, 9, 10],
        [11, 12, 13, 14, 15]
    ]
    return run_test("Тест 2 (уже правильная)", N, M, matrix)


# Тест 3: Пример из условия
def test3():
    N, M = 4, 5
    matrix = [
        [10, 7, 9, 8, 6],
        [15, 12, 14, 13, 11],
        [20, 17, 19, 18, 16],
        [5, 2, 4, 3, 1]
    ]
    return run_test("Тест 3 (пример из условия)", N, M, matrix)


# Тест 4: Только строки перепутаны
def test4():
    N, M = 3, 3
    matrix = [
        [7, 8, 9],
        [1, 2, 3],
        [4, 5, 6]
    ]
    return run_test("Тест 4 (только строки)", N, M, matrix)


# Тест 5: Только столбцы перепутаны
def test5():
    N, M = 2, 4
    matrix = [
        [1, 4, 3, 2],
        [5, 8, 7, 6]
    ]
    return run_test("Тест 5 (только столбцы)", N, M, matrix)


# Тест 6: Матрица 1x1
def test6():
    N, M = 1, 1
    matrix = [[1]]
    return run_test("Тест 6 (1x1)", N, M, matrix)


# Тест 7: Случайная перестановка
def test7():
    N, M = 2, 3
    matrix = [
        [4, 5, 6],
        [1, 2, 3]
    ]
    return run_test("Тест 7 (случайная)", N, M, matrix)


def run_all_tests():
    """Запускает все тесты"""
    tests = [test1, test2, test3, test4, test5, test6, test7]
    passed = 0

    print("=" * 50)
    print("ЗАПУСК АВТОТЕСТОВ")
    print("=" * 50)

    for test in tests:
        try:
            correct, ops = test()
            if correct:
                passed += 1
                print("✅ ТЕСТ ПРОЙДЕН")
            else:
                print("❌ ТЕСТ НЕ ПРОЙДЕН")
        except Exception as e:
            print(f"❌ ОШИБКА В ТЕСТЕ: {e}")

    print("\n" + "=" * 50)
    print(f"ИТОГ: {passed}/{len(tests)} тестов пройдено")
    print("=" * 50)

run_all_tests()

ЗАПУСК АВТОТЕСТОВ

=== Тест 1 (пример из условия) ===
Входная матрица:
4 3
2 1
[memoryit] Function run_solution peak python allocation: 1.35 KiB
[memoryit] Function run_solution peak RSS delta: 0.00 KiB
[timeit] Function run_solution took total 0.000342 seconds

Операции (2):
R 1 2
C 1 2
[memoryit] Function apply_operations peak python allocation: 1.35 KiB
[memoryit] Function apply_operations peak RSS delta: 0.00 KiB
[timeit] Function apply_operations took total 0.000168 seconds

Результирующая матрица:
1 2
3 4
[memoryit] Function is_correct_final_matrix peak python allocation: 1.35 KiB
[memoryit] Function is_correct_final_matrix peak RSS delta: 0.00 KiB
[timeit] Function is_correct_final_matrix took total 0.000141 seconds

Корректность: True
✅ ТЕСТ ПРОЙДЕН

=== Тест 2 (уже правильная) ===
Входная матрица:
1 2 3 4 5
6 7 8 9 10
11 12 13 14 15
[memoryit] Function run_solution peak python allocation: 1.35 KiB
[memoryit] Function run_solution peak RSS delta: 0.00 KiB
[timeit] Function run_

In [ ]:
Задача 23. Пирамидальная сортировка
Ограничение по времени – 1.5 с. Ограничение по памяти – 256 Mb.
В данной задаче необходимо реализовать сортировку кучей. При этом кучу
необходимо реализовать самостоятельно, использовать имеющиеся в языке реализации
нельзя. Сначала рекомендуется решить задачи про просеивание вниз и вверх.
Тимофей решил организовать соревнование по спортивному программированию,
чтобы найти талантливых стажёров. Задачи подобраны, участники зарегистрированы,
тесты написаны. Осталось придумать, как в конце соревнования будет определяться
победитель.
Каждый участник имеет уникальный логин. Когда соревнование закончится, к нему
будут привязаны два показателя: количество решённых задач Pi

и размер штрафа Fi
.

Штраф начисляется за неудачные попытки и время, затраченное на задачу.
Тимофей решил сортировать таблицу результатов следующим образом: при
сравнении двух участников выше будет идти тот, у которого решено больше задач. При
равенстве числа решённых задач первым идёт участник с меньшим штрафом. Если же и
штрафы совпадают, то первым будет тот, у которого логин идёт раньше в алфавитном
(лексикографическом) порядке.
Тимофей заказал толстовки для победителей и накануне поехал за ними в магазин.
В своё отсутствие он поручил вам реализовать алгоритм сортировки кучей (англ. Heapsort)
для таблицы результатов.
Формат входных данных:
В первой строке задано число участников n, l ≤ n ≤ 100000 .
В каждой из следующих и строк задана информация про одного из участников.
i -й участник описывается тремя параметрами:
− уникальным логином (строкой из маленьких латинских букв длиной не более
20)

− числом решенных задач Pi
− Штрафом Fi
Fi и Pi
- целые числа, лежащие в диапазоне от 0 до 109
.

Формат выходных данных:
Для отсортированного списка участников выведите по порядку их логины по одному
в строке.

In [50]:
def _is_better_result(left: tuple[str, int, int], right: tuple[str, int, int]) -> bool:
    if left[1] != right[1]:  #  больше задач — выше
        return left[1] > right[1]
    if left[2] != right[2]:  #  при равенстве сравниваем
        return left[2] < right[2]
    return left[0] < right[0]  # дальше идёт алф порядок


def _sift_down(heap: list[tuple[str, int, int]], start: int, size: int) -> None:
    parent = start
    while True:
        left = 2 * parent + 1  # левый потомок
        right = left + 1       # правый потомок
        best = parent
        
        # сравниваем с левым потомком
        if left < size and _is_better_result(heap[left], heap[best]):
            best = left
        # сравниваем с правым потомком  
        if right < size and _is_better_result(heap[right], heap[best]):
            best = right
        
        # если родитель лучше обоих потомков - выходим
        if best == parent:
            break
            
        # меняем родителя с лучшим потомком
        heap[parent], heap[best] = heap[best], heap[parent]
        parent = best


@timeit
@memoryit
def heap_sort_results(participants: list[tuple[str, int, int]]) -> list[str]:
    heap = participants[:]  # копия входа
    size = len(heap)
    
    # меняем кучи
    for idx in range(size // 2, -1, -1):
        _sift_down(heap, idx, size)
    
    result: list[str] = []
    
    # сортировка
    for end in range(size - 1, -1, -1):
        # перемещаем лучший элемент в конец
        heap[0], heap[end] = heap[end], heap[0]
        result.append(heap[end][0])  # забираем имя участника
        _sift_down(heap, 0, end)    # восстанавливаем кучу
    return result


contestants = [
    ("alla", 4, 100),
    ("gena", 6, 1000),
    ("gosha", 2, 90),
    ("rita", 2, 90),
    ("timofey", 4, 80),
]
print(heap_sort_results(contestants))
print(heap_sort_results([("aaa", 1, 5), ("bbb", 1, 5), ("ccc", 2, 7)]))
print(heap_sort_results([("x", 0, 0), ("y", 0, 0)]))

[memoryit] Function heap_sort_results peak python allocation: 1.35 KiB
[memoryit] Function heap_sort_results peak RSS delta: 0.00 KiB
[timeit] Function heap_sort_results took total 0.000345 seconds
['gena', 'timofey', 'alla', 'gosha', 'rita']
[memoryit] Function heap_sort_results peak python allocation: 1.35 KiB
[memoryit] Function heap_sort_results peak RSS delta: 0.00 KiB
[timeit] Function heap_sort_results took total 0.000158 seconds
['ccc', 'aaa', 'bbb']
[memoryit] Function heap_sort_results peak python allocation: 1.35 KiB
[memoryit] Function heap_sort_results peak RSS delta: 0.00 KiB
[timeit] Function heap_sort_results took total 0.000137 seconds
['x', 'y']


In [ ]:
Задача 25. Куча ли?

Структуру данных «куча», или, более конкретно, «неубывающая пирамида», можно
реализовать на основе массива. Для этого должно выполнятся основное свойство
неубывающей пирамиды, которое заключается в том, что для каждого 1≤i≤n выполняются
условия:
• если 2i≤n, то a[i]≤a[2i];
• если 2i+1≤n, то a[i]≤a[2i+1].

Дан массив целых чисел. Определите, является ли он неубывающей пирамидой.
Формат входных данных:
Первая строка входного файла содержит целое число n (1≤n≤106). Вторая строка
содержит n целых чисел, по модулю не превосходящих 2⋅109.
Формат выходных данных:
Выведите «YES», если массив является неубывающей пирамидой, и «NO» в противном
случае.

In [52]:
@timeit
@memoryit
def is_heap_property(values: list[int]) -> str:
    n = len(values)
    for idx in range(n):
        left = 2 * idx + 1  # левый child в массиве
        right = left + 1  # правый child
        if left < n and values[idx] > values[left]:
            return "NO"
        if right < n and values[idx] > values[right]:
            return "NO"
    return "YES"


print(is_heap_property([1, 3, 5, 7, 9]))
print(is_heap_property([5, 4, 3, 2, 1]))
print(is_heap_property([2]))
print(is_heap_property([2, 2, 2, 2]))

[memoryit] Function is_heap_property peak python allocation: 1.35 KiB
[memoryit] Function is_heap_property peak RSS delta: 0.00 KiB
[timeit] Function is_heap_property took total 0.000311 seconds
YES
[memoryit] Function is_heap_property peak python allocation: 1.35 KiB
[memoryit] Function is_heap_property peak RSS delta: 0.00 KiB
[timeit] Function is_heap_property took total 0.000314 seconds
NO
[memoryit] Function is_heap_property peak python allocation: 1.35 KiB
[memoryit] Function is_heap_property peak RSS delta: 0.00 KiB
[timeit] Function is_heap_property took total 0.000171 seconds
YES
[memoryit] Function is_heap_property peak python allocation: 1.35 KiB
[memoryit] Function is_heap_property peak RSS delta: 0.00 KiB
[timeit] Function is_heap_property took total 0.000130 seconds
YES


In [ ]:
Задача 30. Вложенные отрезки
Ограничение времени: 1 секунда. Ограничение памяти: 64 МБ.
На прямой лежат n отрезков. Для каждой пары отрезков известно, что они либо не
имеют общих точек, либо все точки одного из них также принадлежат и другому отрезку.
Дано m запросов. Каждый запрос представляет собой точку на прямой. Найдите для
каждого запроса отрезок минимальной длины, которому принадлежит эта точка.
Формат входных данных:
В первой строке записано целое число n — количество отрезков (1 ≤ n ≤ 105
). i-я из
следующих n строк содержит целые числа ai и bi — координаты концов i-го отрезка (1
≤ ai < bi ≤ 109

). Отрезки упорядочены по неубыванию ai, а при ai = aj — по убыванию
длины. Совпадающих отрезков нет. В следующей строке записано целое число m —
количество запросов (1 ≤ m ≤ 105

). В j-й из следующих m строк записано целое число cj —

координата точки (1 ≤ cj ≤ 109

). Запросы упорядочены по неубыванию cj.

Формат выходных данных:
Для каждого запроса выведите номер искомого отрезка в отдельной строке. Если
точка не принадлежит ни одному отрезку, выведите «-1». Отрезки пронумерованы числами
от 1 до n в том порядке, в котором они перечислены во входных данных.

In [54]:
@timeit
@memoryit
def nested_segments(segments: list[tuple[int, int]], points: list[int]) -> list[int]:
    if not segments or not points:
        return [-1] * len(points)

    answers = [-1] * len(points)
    stack: list[tuple[int, int]] = []  # (конец отрезка, индекс отрезка)
    seg_idx = 0  # seg_idx - указатель на текущий отрезок
    total_segments = len(segments)

    for point_idx, point in enumerate(points):
        #  добавляем все отрезки начинающиеся до текущей точки
        while seg_idx < total_segments and segments[seg_idx][0] <= point:
            start, end = segments[seg_idx]
            while stack and stack[-1][0] < start:
                stack.pop()
            stack.append((end, seg_idx + 1))
            seg_idx += 1

        # удаляем отрезки, которые не содержат текущую точку
        while stack and stack[-1][0] < point:
            stack.pop()

        # записываем результат
        if stack:
            answers[point_idx] = stack[-1][1]
        else:
            answers[point_idx] = -1
    return answers


test_segments = [(1, 10), (2, 5), (6, 9)]
test_points = [1, 3, 6, 11]
print(nested_segments(test_segments, test_points))
print(nested_segments([(1, 4), (2, 3)], [0, 2, 4]))
print(nested_segments([(1, 5), (5, 10)], [5, 6, 1]))



[memoryit] Function nested_segments peak python allocation: 1.35 KiB
[memoryit] Function nested_segments peak RSS delta: 0.00 KiB
[timeit] Function nested_segments took total 0.000357 seconds
[1, 2, 3, -1]
[memoryit] Function nested_segments peak python allocation: 1.35 KiB
[memoryit] Function nested_segments peak RSS delta: 0.00 KiB
[timeit] Function nested_segments took total 0.000190 seconds
[-1, 2, 1]
[memoryit] Function nested_segments peak python allocation: 1.35 KiB
[memoryit] Function nested_segments peak RSS delta: 0.00 KiB
[timeit] Function nested_segments took total 0.000136 seconds
[2, 2, 2]
